残差神经网络思想：只有当较复杂的函数类包含较小的函数类时，我们才能确保提高它们的性能。 对于深度神经网络，如果我们能将新添加的层训练成恒等映射（identity function）
，新模型和原模型将同样有效。 同时，由于新模型可能得出更优的解来拟合训练数据集，因此添加层似乎更容易降低训练误差。

 残差网络的核心思想是：每个附加层都应该更容易地包含原始函数作为其元素之一。 于是，残差块（residual blocks）便诞生了，这个设计对如何建立深层神经网络产生了深远的影响。

In [ ]:
from torch import nn


class Residual(nn.Module):
    def __init__(self, in_features, num_channels,use_1x1conv=False,strides=1):
        super(Residual, self).__init__()
        self.conv1=nn.Conv2d(in_channels=in_features,out_channels=num_channels,kernel_size=3,stride=strides,padding=1)
        self.conv2=nn.Conv2d(in_channels=num_channels,out_channels=num_channels,kernel_size=3,stride=1,padding=1)

        if use_1x1conv:
            self.conv3=nn.Conv2d(in_channels=num_channels,out_channels=num_channels,kernel_size=3,stride=1,padding=1)